In [14]:
import requests
import zipfile
import os

class BKT(object):
    def __init__(self, 
                 hmm_folder='./hmm-scalable-818d905234a8600a8e3a65bb0f7aa4cf06423f1a', 
                 git_commit='818d905234a8600a8e3a65bb0f7aa4cf06423f1a'):
        
        # Git commit to download hmm-scalable
        self.git_commit = git_commit
        # Set HMM-scalable folder.
        self.hmm_folder = hmm_folder
    
    def download(self):
        """  This implementation is a wrapper around the 
        HMM-scalable tool ( http://yudelson.info/hmm-scalable).
        This function will download the original implementation."""
        
        # Download zipfile from GitHub
#         results = requests.get('https://github.com/myudelson/hmm-scalable/archive/master.zip')
        results = requests.get('https://github.com/myudelson/hmm-scalable/archive/%s.zip' % self.git_commit)
        with open('/tmp/hmm-scalable.zip', 'wb') as f:
            f.write(results.content)
            
        # Extract zipfile
        file = zipfile.ZipFile('/tmp/hmm-scalable.zip')
        file.extractall(path='.')
        
    
    def fit(self, data, q_matrix, solver='bw', iterations=200):
        """ Fit BKT model to data. 
        As of July 2019, just default parameters are allowed.
        
        Parameters
        ----------
        data : {array-like}, shape (n_steps, 3)
            Sequence of students steps. Each of the three dimensions are:
            Observed outcome: 0 for fail and 1 for success
            Student id: student unique identifier
            Question id: question id in q_matrix
            
        q_matrix: matrix, shape (n_questions, n_concepts)
            Each row is a question and each column a concept.
            If the concept is present in the question, the 
            correspondent cell should contain 1, otherwise, 0.
            
        solver: string, optional
            Algorithm used to fit the BKT model. Available solvers are:
            'bw': Baum-Welch (default)
            'gd': Gradient Descent
            'cgd_pr': Conjugate Gradient Descent (Polak-Ribiere)
            'cgd_fr': Conjugate Gradient Descent (Fletcher–Reeves)
            'cgd_hs': Conjugate Gradient Descent (Hestenes-Stiefel)
            
        iterations: integer, optional
            Maximum number of iterations
        
        Returns
        -------
        self : object
        
        Notes
        -----
        This is a wrapper around the HMM-scalable tool (http://yudelson.info/hmm-scalable)
        """
        os.system()

### Unit tests

In [37]:
import unittest
import os

class TestBKT(unittest.TestCase):
    def test_download(self):
        """ Testing HMM-scalable download """
        model = BKT()
        model.download()
        
        # Check if directory exists and it contains items
        self.assertGreater(len(os.listdir(model.hmm_folder)), 1)

In [36]:
TestBKT().test_download()

67


### Generate sample data

In [38]:
# p(L0)
pi = [0.26, 0.74]

# p(T)
A = [[1, 0], [0.17, 0.83]]

# p(S) and p(G)
B = [[0.7, 0.3], [0.13, 0.87]]

In [71]:
import numpy as np
class GS(object):
    def __init__(self):
        self.emission = B
        self.priors = pi
        
    def random_MN_draw(self, n, probs):
        """ get X random draws from the multinomial distribution whose probability is given by 'probs' """
        mn_draw = np.random.multinomial(n,probs) # do 1 multinomial experiment with the given probs with probs= [0.5,0.5], this is a coin-flip
        print(mn_draw)
        return np.where(mn_draw == 1)[0][0] # get the index of the state drawn e.g. 0, 1, etc.

    def simulate(self, nSteps):
        """ given an HMM = (A, B1, B2 pi), simulate state and observation sequences """
        lenB = len(self.emission)
        observations = np.zeros((lenB, nSteps), dtype=np.int) # array of zeros
        states = np.zeros(nSteps)
        states[0] = self.random_MN_draw(1, self.priors) # appoint the first state from the prior dist
        
        for i in range(0,lenB): # initialise observations[i,0] for all observerd variables
            observations[i,0] = self.random_MN_draw(1, self.emission[i][states[0],:]) #ith variable array, states[0]th row

#         for t in range(1,nSteps): # loop through t
#             states[t] = self.random_MN_draw(1, self.transition[states[t-1],:]) # given prev state (t-1) pick what row of the A matrix to use

#             for i in range(0,lenB): # loop through the observed variable for each t
#                 observations[i,t] = self.random_MN_draw(1, self.emission[i][states[t],:]) # given current state t, pick what row of the B matrix to use

#         return observations,states

In [77]:
GS().simulate(5)

[0 1]


TypeError: list indices must be integers or slices, not tuple